In [1]:
import collections
import numpy as np
import pandas as pd
import re
# get filtered_comments
comments = pd.read_csv("filtered_comment_lgbtq.csv", encoding='latin-1')
comments.shape

(105260, 3)

In [2]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

c:\Users\jeffr\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# search terms
General = ['LGBT', 'LGBTQ', 'LGBTQ+', 'LGBTQIA+', 'homosexuality' , 'homosexual', 'homo', 'sexual orientation', 'gender dysphoria', 'gender identity' ]
Gay = ['Gay', 'ah gua', 'ah qua'] 
Lesbian = ['Lesbian', 'les', 'sapphic']
Bisexual = ['Bisexual']
Transgender =['Transgender', 'tranny', 'ladyboy', 'trans', 'sex reassignment'] 
Queer = ['Queer', 'non-binary', 'non binary']
Same_Sex_Marriage = ['same-sex couples', 'same-sex marriage', 'same sex marriage', 'same-sex partner', 'same sex', 'gay marriage']
LGBTQ_rights = [ 'LGBT right', 'rights', 'IVF', 'adoption', 'housing']  
LGBTQ_mental_wellbeing = ['depression', 'suicide', 'mental health', 'mental illness']
Diseases = ['HIV', 'HPV', 'syphilis', 'gonorrhoea', 'AIDS', 'diseases']
Pride_Month = ['Pink dot', 'Wear White', 'Pride Month', 'pride parades']
Anti_LGBTQ_Law = ['Anti-LGBTQ law', 'anti-LGBT laws', 'anti-LGBTQ laws', 'S377a', 'Section377a', 'section 377A', '377a', 'anti-gay law', 
                'Anti LGBTQ law', 'anti LGBT laws', 'anti LGBTQ laws', 'anti gay law', '377a penal code']
                


search_terms = []
search_terms.extend(General)
search_terms.extend(Gay)
search_terms.extend(Lesbian)
search_terms.extend(Bisexual)
search_terms.extend(Transgender)
search_terms.extend(Queer)
search_terms.extend(Same_Sex_Marriage)
search_terms.extend(LGBTQ_rights)
search_terms.extend(LGBTQ_mental_wellbeing)
search_terms.extend(Diseases)
search_terms.extend(Pride_Month)
search_terms.extend(Anti_LGBTQ_Law)
search_terms


['LGBT',
 'LGBTQ',
 'LGBTQ+',
 'LGBTQIA+',
 'homosexuality',
 'homosexual',
 'homo',
 'sexual orientation',
 'gender dysphoria',
 'gender identity',
 'Gay',
 'ah gua',
 'ah qua',
 'Lesbian',
 'les',
 'sapphic',
 'Bisexual',
 'Transgender',
 'tranny',
 'ladyboy',
 'trans',
 'sex reassignment',
 'Queer',
 'non-binary',
 'non binary',
 'same-sex couples',
 'same-sex marriage',
 'same sex marriage',
 'same-sex partner',
 'same sex',
 'gay marriage',
 'LGBT right',
 'rights',
 'IVF',
 'adoption',
 'housing',
 'depression',
 'suicide',
 'mental health',
 'mental illness',
 'HIV',
 'HPV',
 'syphilis',
 'gonorrhoea',
 'AIDS',
 'diseases',
 'Pink dot',
 'Wear White',
 'Pride Month',
 'pride parades',
 'Anti-LGBTQ law',
 'anti-LGBT laws',
 'anti-LGBTQ laws',
 'S377a',
 'Section377a',
 'section 377A',
 '377a',
 'anti-gay law',
 'Anti LGBTQ law',
 'anti LGBT laws',
 'anti LGBTQ laws',
 'anti gay law',
 '377a penal code']

In [4]:
def lgbtq_related(text):
    cnt = 0
    for i in process.extract(text, choices=search_terms, limit=64):
        if i[1] >= 50:
            cnt += 1
    return cnt

In [5]:
comments['comment'] = comments['comment'].apply(lambda x: str(x) if not pd.isna(x) else "")
comments['contains_only_digits'] = comments['comment'].apply(lambda x: bool(re.match('^[0-9]+$', x)))
comments = comments[~comments['contains_only_digits']]
comments['score'] = comments['comment'].apply(lgbtq_related)

In [6]:
df = comments.drop('contains_only_digits', axis=1)
df = df[df['score']>2]
df.shape
df.to_csv('fuzzy_filtered_data.csv', index=False, header=True)

In [7]:
import collections
import numpy as np
import pandas as pd
import nltk
import re

from nltk.tokenize import word_tokenize, sent_tokenize
from argparse import Namespace

In [8]:
args = Namespace(                                 
    filtered_dataset_csv="fuzzy_filtered_data.csv",
    output_filtered_csv="fuzzy_filtered_data_processed_test.csv",
)

In [9]:
# Read raw data
filtered_comments = pd.read_csv(args.filtered_dataset_csv)

In [10]:
filtered_comments.head()

,comment,create_utc,community,score
0,i think it s rather ironic that anti lgbtq gro...,2023-09-11 18:56:33,Reddit,19
1,here https transgendersg . com response to dr...,2023-09-11 21:55:35,Reddit,23
2,conservatism largely have no real data or fact...,2023-09-12 06:29:11,Reddit,12
3,i find it funny that the counsellor is obsesse...,2023-09-12 00:11:57,Reddit,10
4,"sometimes , i feel hor . . what if , really wh...",2023-09-11 20:35:04,Reddit,9


In [11]:
def count_words(text):
    # Tokenize the text into words and return the word count
    return len(word_tokenize(text))

# Calculate the word count for each row
filtered_comments['Word Count'] = filtered_comments.comment.apply(count_words)


# Drop rows with less than 3 words
filtered_comments = filtered_comments[filtered_comments['Word Count'] >= 3]

# Remove the 'Word Count' column if you don't need it anymore
filtered_comments.drop('Word Count', axis=1, inplace=True)

In [12]:
filtered_comments.head()

,comment,create_utc,community,score
0,i think it s rather ironic that anti lgbtq gro...,2023-09-11 18:56:33,Reddit,19
1,here https transgendersg . com response to dr...,2023-09-11 21:55:35,Reddit,23
2,conservatism largely have no real data or fact...,2023-09-12 06:29:11,Reddit,12
3,i find it funny that the counsellor is obsesse...,2023-09-12 00:11:57,Reddit,10
4,"sometimes , i feel hor . . what if , really wh...",2023-09-11 20:35:04,Reddit,9


In [13]:
# df to csv file with header
filtered_comments.to_csv(args.output_filtered_csv, index=False, header=True) 

In [14]:
filtered_comments.shape

(16718, 4)